In [2]:
from base64 import b64encode, b64decode
from time import time, sleep
import math, random
import numpy as np
from Crypto.Cipher import AES

In [3]:
def xor(*args):
    assert len(args) >= 2 and min(map(len, args)) == max(map(len, args))
    if len(args) == 2:
        a,b = args
        return bytes([x^y for (x, y) in zip(a,b)])
    else:
        return xor(args[0], xor(*args[1:]))
    
def key_xor(msg, key):
    stream = key * (len(msg) // len(key)) + key[:(len(msg) % len(key))]
    return xor(msg, stream)

def pad(msg, block_size):
    extra_bytes = block_size - len(msg) % block_size 
    return msg + bytes([extra_bytes] * extra_bytes)

class PaddingException(Exception):
    pass

def unpad(msg):
    if not msg or not 0 < msg[-1] <= 16 or msg[-msg[-1]:] != bytes([msg[-1]] * msg[-1]):
        raise PaddingException
    return msg[:-msg[-1]]

def random_bytes(len):
    return bytes([random.randint(0,255) for _ in range(len)])

def cbc_encrypt(msg, IV, cipher):
    assert len(msg) % 16 == 0
    assert len(IV) == 16
    out = bytes([])
    curr_block = IV[:]
    for i in range(len(msg) // 16):
        curr_block = xor(curr_block, msg[i*16:(i+1)*16])
        curr_block = cipher.encrypt(curr_block)
        out += curr_block
    assert len(out) == len(msg)
    return out

def cbc_decrypt(ciphtxt, IV, cipher):
    assert len(ciphtxt) % 16 == 0
    assert len(IV) == 16
    out = bytes([])
    curr_block = IV[:]
    for i in range(len(ciphtxt) // 16):
        dec_block = cipher.decrypt(ciphtxt[i*16:(i+1)*16])
        dec_block = xor(curr_block, dec_block)
        out += dec_block
        curr_block = ciphtxt[i*16:(i+1)*16]
    assert len(out) == len(ciphtxt)
    return out

### Challenge 17

In [4]:
msgs = [
    'MDAwMDAwTm93IHRoYXQgdGhlIHBhcnR5IGlzIGp1bXBpbmc=',
    'MDAwMDAxV2l0aCB0aGUgYmFzcyBraWNrZWQgaW4gYW5kIHRoZSBWZWdhJ3MgYXJlIHB1bXBpbic=',
    'MDAwMDAyUXVpY2sgdG8gdGhlIHBvaW50LCB0byB0aGUgcG9pbnQsIG5vIGZha2luZw==',
    'MDAwMDAzQ29va2luZyBNQydzIGxpa2UgYSBwb3VuZCBvZiBiYWNvbg==',
    'MDAwMDA0QnVybmluZyAnZW0sIGlmIHlvdSBhaW4ndCBxdWljayBhbmQgbmltYmxl',
    'MDAwMDA1SSBnbyBjcmF6eSB3aGVuIEkgaGVhciBhIGN5bWJhbA==',
    'MDAwMDA2QW5kIGEgaGlnaCBoYXQgd2l0aCBhIHNvdXBlZCB1cCB0ZW1wbw==',
    'MDAwMDA3SSdtIG9uIGEgcm9sbCwgaXQncyB0aW1lIHRvIGdvIHNvbG8=',
    'MDAwMDA4b2xsaW4nIGluIG15IGZpdmUgcG9pbnQgb2g=',
    'MDAwMDA5aXRoIG15IHJhZy10b3AgZG93biBzbyBteSBoYWlyIGNhbiBibG93',
]
msgs = [b64decode(str) for str in msgs]

In [5]:
key = random_bytes(16)
cipher = AES.new(key, AES.MODE_ECB)

def encryption_oracle():
    msg = msgs[random.randint(1, len(msgs)-1)]
    IV = random_bytes(16)
    return cbc_encrypt(pad(msg, 16), IV, cipher), IV

def padding_oracle(ciphtxt, IV):
    try:
        unpad(cbc_decrypt(ciphtxt, IV, cipher))
        return True
    except PaddingException:
        return False

In [6]:
ciphtxt, IV = encryption_oracle()
ciphtxt = IV + ciphtxt

In [7]:
final_msg = b''

for block in range(1, len(ciphtxt) // 16):
    msg = b''
    for offset in range(15, -1, -1):
        pad_num = 16 - offset
        while True:
            for c in range(256):
                prev_block = random_bytes(offset) + bytes([c]) + \
                             xor(ciphtxt[(block-1)*16+offset+1:block*16], msg, bytes([pad_num] * len(msg)))
                curr_block = ciphtxt[block*16:(block+1)*16]
                if padding_oracle(curr_block, prev_block):
                    msg = bytes([c ^ pad_num ^ ciphtxt[(block-1)*16+offset]]) + msg
                    break
            if len(msg) + offset + 1 > 16:
                break
    final_msg += msg

In [8]:
final_msg

b"000003Cooking MC's like a pound of bacon\x08\x08\x08\x08\x08\x08\x08\x08"

### Challenge 18

In [9]:
key = "YELLOW SUBMARINE"
nonce = bytes(8)
cipher = AES.new(key, AES.MODE_ECB)

def ctr_keystream(nonce, count, cipher):
    return cipher.encrypt(nonce + count.to_bytes(8, byteorder='little'))

def ctr(msg, nonce, cipher):
    out = b''
    count = 0
    for i in range(len(msg) // 16):
        out += xor(ctr_keystream(nonce, count, cipher), msg[i*16:(i+1)*16])
        count += 1
    out += xor(ctr_keystream(nonce, count, cipher)[:len(msg)%16], msg[len(msg)-len(msg)%16:])
    return out

In [10]:
ciphtxt = b'L77na/nrFsKvynd6HzOoG7GHTLXsTVu9qvY/2syLXzhPweyyMTJULu/6/kXX0KSvoOLSFQ=='
ciphtxt = b64decode(ciphtxt)
ctr(ciphtxt, nonce, cipher)

b"Yo, VIP Let's kick it Ice, Ice, baby Ice, Ice, baby "

### Challenge 19

In [11]:
freq = {' ': 20, 
        'e': 12.70, 
        't': 9.06, 
        'a': 8.17, 
        'o': 7.51, 
        'i': 6.97, 
        'n': 6.75, 
        's': 6.33, 
        'h': 6.09, 
        'r': 5.99, 
        'd': 4.25, 
        'l': 4.03, 
        'c': 2.78, 
        'u': 2.76, 
        'm': 2.41, 
        'w': 2.36, 
        'f': 2.23, 
        'g': 2.02, 
        'y': 1.97, 
        'p': 1.93, 
        'b': 1.29, 
        'v': 0.98, 
        'k': 0.77, 
        'j': 0.15, 
        'x': 0.15, 
        'q': 0.10, 
        'z': 0.07}

def get_msg_score(msg):
    return sum([(0 if chr(b).lower() not in freq else freq[chr(b).lower()]) for b in msg])

In [12]:
msgs = \
    '''SSBoYXZlIG1ldCB0aGVtIGF0IGNsb3NlIG9mIGRheQ==
    Q29taW5nIHdpdGggdml2aWQgZmFjZXM=
    RnJvbSBjb3VudGVyIG9yIGRlc2sgYW1vbmcgZ3JleQ==
    RWlnaHRlZW50aC1jZW50dXJ5IGhvdXNlcy4=
    SSBoYXZlIHBhc3NlZCB3aXRoIGEgbm9kIG9mIHRoZSBoZWFk
    T3IgcG9saXRlIG1lYW5pbmdsZXNzIHdvcmRzLA==
    T3IgaGF2ZSBsaW5nZXJlZCBhd2hpbGUgYW5kIHNhaWQ=
    UG9saXRlIG1lYW5pbmdsZXNzIHdvcmRzLA==
    QW5kIHRob3VnaHQgYmVmb3JlIEkgaGFkIGRvbmU=
    T2YgYSBtb2NraW5nIHRhbGUgb3IgYSBnaWJl
    VG8gcGxlYXNlIGEgY29tcGFuaW9u
    QXJvdW5kIHRoZSBmaXJlIGF0IHRoZSBjbHViLA==
    QmVpbmcgY2VydGFpbiB0aGF0IHRoZXkgYW5kIEk=
    QnV0IGxpdmVkIHdoZXJlIG1vdGxleSBpcyB3b3JuOg==
    QWxsIGNoYW5nZWQsIGNoYW5nZWQgdXR0ZXJseTo=
    QSB0ZXJyaWJsZSBiZWF1dHkgaXMgYm9ybi4=
    VGhhdCB3b21hbidzIGRheXMgd2VyZSBzcGVudA==
    SW4gaWdub3JhbnQgZ29vZCB3aWxsLA==
    SGVyIG5pZ2h0cyBpbiBhcmd1bWVudA==
    VW50aWwgaGVyIHZvaWNlIGdyZXcgc2hyaWxsLg==
    V2hhdCB2b2ljZSBtb3JlIHN3ZWV0IHRoYW4gaGVycw==
    V2hlbiB5b3VuZyBhbmQgYmVhdXRpZnVsLA==
    U2hlIHJvZGUgdG8gaGFycmllcnM/
    VGhpcyBtYW4gaGFkIGtlcHQgYSBzY2hvb2w=
    QW5kIHJvZGUgb3VyIHdpbmdlZCBob3JzZS4=
    VGhpcyBvdGhlciBoaXMgaGVscGVyIGFuZCBmcmllbmQ=
    V2FzIGNvbWluZyBpbnRvIGhpcyBmb3JjZTs=
    SGUgbWlnaHQgaGF2ZSB3b24gZmFtZSBpbiB0aGUgZW5kLA==
    U28gc2Vuc2l0aXZlIGhpcyBuYXR1cmUgc2VlbWVkLA==
    U28gZGFyaW5nIGFuZCBzd2VldCBoaXMgdGhvdWdodC4=
    VGhpcyBvdGhlciBtYW4gSSBoYWQgZHJlYW1lZA==
    QSBkcnVua2VuLCB2YWluLWdsb3Jpb3VzIGxvdXQu
    SGUgaGFkIGRvbmUgbW9zdCBiaXR0ZXIgd3Jvbmc=
    VG8gc29tZSB3aG8gYXJlIG5lYXIgbXkgaGVhcnQs
    WWV0IEkgbnVtYmVyIGhpbSBpbiB0aGUgc29uZzs=
    SGUsIHRvbywgaGFzIHJlc2lnbmVkIGhpcyBwYXJ0
    SW4gdGhlIGNhc3VhbCBjb21lZHk7
    SGUsIHRvbywgaGFzIGJlZW4gY2hhbmdlZCBpbiBoaXMgdHVybiw=
    VHJhbnNmb3JtZWQgdXR0ZXJseTo=
    QSB0ZXJyaWJsZSBiZWF1dHkgaXMgYm9ybi4='''.split('\n')
msgs = [b64decode(str) for str in msgs]

In [13]:
ciphtxts = [ctr(msg, nonce, cipher) for msg in msgs]

In [14]:
keystream = bytearray(b'')

for i in range(max([len(msg) for msg in msgs])):
    best_ch = None
    best_score = 0
    for ch in range(256):
        score = 0
        for ciphtxt in ciphtxts:
            if i < len(ciphtxt) and chr(ciphtxt[i] ^ ch) in freq:
                score += freq[chr(ciphtxt[i] ^ ch)]
        if score > best_score:
            best_score = score
            best_ch = ch
    keystream.append(best_ch)
    
keystream = bytes(keystream)

for ciphtxt in ciphtxts:
    print(key_xor(ciphtxt, keystream))

b'i have met them at close of dau'
b'coming with vivid faces'
b'from counter or desk among greu'
b'eighteenth-century houses.'
b'i have passed with a nod of thidhi  '
b'or polite meaningless words,'
b'or have lingered awhile and sae '
b'polite meaningless words,'
b'and thought before I had done'
b'of a mocking tale or a gibe'
b'to please a companion'
b'around the fire at the club,'
b'being certain that they and I'
b'but lived where motley is worn6'
b'all changed, changed utterly:'
b'a terrible beauty is born.'
b"that woman's days were spent"
b'in ignorant good will,'
b'her nights in argument'
b'until her voice grew shrill.'
b'what voice more sweet than her\x7f'
b'when young and beautiful,'
b'she rode to harriers?'
b'this man had kept a school'
b'and rode our winged horse.'
b'this other his helper and frieb '
b'was coming into his force;'
b'he might have won fame in the i*d '
b'so sensitive his nature seemed '
b'so daring and sweet his thoughxj'
b'this other man I had dreamed'
b'a drunk

### Challenge 20

In [15]:
data = bytearray()
with open('20.txt', 'r') as file:
    ciphtxts = [ctr(b64decode(line), nonce, cipher) for line in file.readlines()]
    max_len = max([len(ciphtxt) for ciphtxt in ciphtxts])
    for ciphtxt in ciphtxts:
        data += ciphtxt
        data += random_bytes(max_len - len(ciphtxt))

In [16]:
def best_msg_score(x):
    best_score = -float('inf')
    for c in range(256):
        y = bytes([b ^ c for b in x])
        score = get_msg_score(y)
        if score >= best_score:
            best_letter = chr(c)
            best_score = score
            msg = y
    return best_letter, best_score, msg

In [17]:
blocks = []
for k in range(max_len):
    blocks.append([])
    for i in range(k, len(data), max_len):
        blocks[k].append(data[i])
    blocks[k] = bytes(blocks[k])
        
letters = []
for block in blocks:
    letter, _, _ = best_msg_score(block)
    letters.append(letter)
    
key = bytes([ord(letter) for letter in letters])

for ciphtxt in ciphtxts:
    print(key_xor(ciphtxt, key))

b'N\'m rated "R"...this is a warning, ya better void / Poets are paranoid, DJ\'s D-stroyed'
b"Duz I came back to attack others in spite- / Strike like lightnin', It's quite frightenin'!"
b"Eut don't be afraid in the dark, in a park / Not a scream or a cry, or a bark, more like a spark*"
b"^a tremble like a alcoholic, muscles tighten up / What's that, lighten up! You see a sight but"
b'Tuddenly you feel like your in a horror flick / You grab your heart then wish for tomorrow quick0'
b"Jusic's the clue, when I come your warned / Apocalypse Now, when I'm done, ya gone!"
b"Oaven't you ever heard of a MC-murderer? / This is the death penalty,and I'm servin' a"
b'Ceath wish, so come on, step to this / Hysterical idea for a lyrical professionist!'
b'Ariday the thirteenth, walking down Elm Street / You come in my realm ya get beat!'
b'Shis is off limits, so your visions are blurry / All ya see is the meters at a volume'
b"Serror in the styles, never error-files / Indeed I'm known-your exiled!"

### Challenge 21

In [18]:
def get_lowest_bits(x, n):
    return x & int('0' * (32-n) + '1' * n, 2)

In [19]:
# based on pseudocode from https://en.wikipedia.org/wiki/Mersenne_Twister

w, n, m, r = 32, 624, 397, 31
a = int('9908b0df', 16)
u, d = 11, int('ffffffff', 16)
s, b = 7, int('9d2c5680', 16)
t, c = 15, int('efc60000', 16)
l = 18
f = 1812433253

In [20]:
MT = [0] * n
index = n + 1
lower_mask = (1 << r) - 1
upper_mask = get_lowest_bits(~lower_mask, w)

def seed_mt(seed):
    global MT, index
    index = n
    MT[0] = seed
    for i in range(1, n):
        MT[i] = get_lowest_bits(f * (MT[i-1] ^ (MT[i-1] >> (w-2))) + i, w)
        
def extract_number():
    global MT, index
    if index >= n:
        if index > n:
            raise Exception("Generator was never seeded")
        twist()
    
    y = MT[index]
    y ^= (y >> u) & d
    y ^= (y << s) & b
    y ^= (y << t) & c
    y ^= (y >> l)
    
    index += 1
    return get_lowest_bits(y, w)

def twist():
    global MT, index
    for i in range(n):
        x = MT[i] & upper_mask + MT[(i+1) % n] & lower_mask
        xA = x >> 1
        if x % 2 != 0:
            xA ^= a
        MT[i] = MT[(i+1) % n] ^ xA
    index = 0

In [21]:
seed_mt(1234)
for i in range(10):
    print(extract_number())

1445446450
1764435502
3060188693
1245365347
453264056
1789102384
2923918431
3919004792
2632166984
3571494490


### Challenge 22

In [22]:
def mt_timestamp_oracle(min_wait=4, max_wait=10):
    sleep(random.randint(min_wait, max_wait))
    seed_mt(int(time()))
    sleep(random.randint(min_wait, max_wait))
    return extract_number()

In [23]:
start_time = int(time())
print(start_time)
x = mt_timestamp_oracle()
print(x)
end_time = int(time())
print(end_time)

1545412067
2978197721
1545412081


In [24]:
for seed in range(start_time, end_time+1):
    seed_mt(seed)
    y = extract_number()
    if y == x:
        print(seed)
        break

1545412071


### Challenge 23

In [25]:
def mt_oracle():
    out = []
    seed = random.randint(0, 2**32 - 1)
    seed_mt(seed)
    return [extract_number() for _ in range(624)]
    
mt_out = mt_oracle()

In [26]:
def reverse_rshift_xor(z, s, mask=None):
    y = y_t = 0
    first_s_mask = int('1' * s + '0' * (32-s), 2)
    for i in range(math.ceil(32 / s)):
        if not mask:
            y_t ^= (z & first_s_mask)
        else:
            y_t = (y_t & mask) ^ (z & first_s_mask)
            mask <<= s
        y |= y_t >> (s * i)
        z <<= s
    return y

def reverse_lshift_xor(z, s, mask=None):
    y = y_t = 0
    last_s_mask = int('0' * (32-s) + '1' * s, 2)
    for i in range(math.ceil(32 / s)):
        if not mask:
            y_t ^= (z & last_s_mask)
        else:
            y_t = (y_t & mask) ^ (z & last_s_mask)
            mask >>= s
        y |= y_t << (s * i)
        z >>= s
    return y

def untemper(z):
    z = reverse_rshift_xor(z, l)
    z = reverse_lshift_xor(z, t, c)
    z = reverse_lshift_xor(z, s, b)
    z = reverse_rshift_xor(z, u, d)
    return z

In [27]:
y_test = random.randint(0, 2**32 - 1)
y_test_init = y_test

y_test ^= (y_test >> u) & d
y_test ^= (y_test << s) & b
y_test ^= (y_test << t) & c
y_test ^= (y_test >> l)

assert untemper(y_test) == y_test_init

In [28]:
MT = []
index = 0

for z in mt_out:
    MT.append(untemper(z))
    
for i in range(624):
    assert extract_number() == mt_out[i]

### Challenge 24

In [29]:
def mts_keystream(seed, num_bytes):
    out = bytearray([])
    seed_mt(seed)
    while len(out) < num_bytes:
        num = extract_number()
        out += num.to_bytes(4, byteorder='big')
    return bytes(out[:num_bytes])

In [30]:
def mts_stream_oracle():
    seed = random.randint(0, 2**16-1)
    msg = random_bytes(random.randint(0, 16)) + 14 * b'A'
    return key_xor(msg, mts_keystream(seed, len(msg)))

In [31]:
ciphtxt = mts_stream_oracle()

In [32]:
ptext_start = math.ceil((len(ciphtxt) - 14) / 4) * 4
stream = xor(ciphtxt[ptext_start:ptext_start+8], 8 * b'A')
num1 = int.from_bytes(stream[:4], byteorder='big')
num2 = int.from_bytes(stream[4:], byteorder='big')
for seed in range(2**16):
    seed_mt(seed)
    [extract_number() for _ in range(ptext_start // 4)]
    if extract_number() == num1 and extract_number() == num2:
        print(seed)
        break

60175


TODO: last part of Challenge 24